<a href="https://colab.research.google.com/github/ZhangYHe/MyColabDL_Repo/blob/main/work04_%E9%A2%84%E6%B5%8B%2Bcsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def iou(box1: torch.Tensor, box2:torch.Tensor, isleftT2rightD = True) -> torch.Tensor:
    # box1 的shape为(1, 4), box2的shape为(None, 4)
    # 防止输入错误
    box1 = box1.view(-1,4)
    box2 = box2.view(-1,4)
    box1 = box1.repeat((box2.shape[0], 1))
    if not isleftT2rightD:
        box1 = torch.concat([box1[:,:2] - box1[:,2:4] / 2, box1[:,:2] + box1[:,2:4] / 2], 1).cuda()
        box2 = torch.concat([box2[:,:2] - box2[:,2:4] / 2, box2[:,:2] + box2[:,2:4] / 2], 1).cuda()
    # 交集左上角的点
    lu = torch.max(box1[:, :2], box2[:, :2])
    # 交集右下角的点
    rd = torch.min(box1[:, 2:], box2[:, 2:])
    rectsN = rd - lu
    rectsN[rectsN < 0] = 0#没有重叠区域设置为0
    rectsN = rectsN[:,0] * rectsN[:,1]
    rectsU = (box1[:,2] - box1[:,0]) * (box1[:,3] - box1[:,1]) + (box2[:,2] - box2[:,0]) * (box2[:,3] - box2[:,1])
    return rectsN / (rectsU - rectsN)

In [ ]:
def nms(box: torch.Tensor = None, score: torch.Tensor = None,threshold: float = 0.3) -> None:
    _, sortIndex =  score.sort(0, descending = True)
    res = []
    while sortIndex.size(0):
        if sortIndex.size(0) == 1:
            res.append(sortIndex[0].item())
            break
        res.append(sortIndex[0].item())
        ious = iou(box[sortIndex[0]], box[sortIndex[1:]])
        sortIndex = sortIndex[1:][ious < threshold]
    return  res

In [ ]:
def notMaxSuppression(inputVal, confThres = 0.5):
    # 化为左上角+右下角坐标
    box_corner = inputVal.new(inputVal.shape)
    box_corner[:, :, 0] = inputVal[:, :, 0] - inputVal[:, :, 2] / 2
    box_corner[:, :, 1] = inputVal[:, :, 1] - inputVal[:, :, 3] / 2
    box_corner[:, :, 2] = inputVal[:, :, 0] + inputVal[:, :, 2] / 2
    box_corner[:, :, 3] = inputVal[:, :, 1] + inputVal[:, :, 3] / 2
    inputVal[:, :, :4] = box_corner[:, :, :4]
    output = [None for _ in inputVal]
    for i, prediction in enumerate(inputVal):
        # 置信度与对应的类型
        classP, classType = torch.max(prediction[:, 5:], 1, keepdim=True)
        # 利用置信度进行第一轮筛选
        confMask = (prediction[...,4] * classP[...,0] >= confThres)
        
        prediction = prediction[confMask]
        classP = classP[confMask]
        classType = classType[confMask]
 
        if not prediction.shape[0]:
            continue
        # 整合数据
        prediction = torch.cat([prediction[:,:5], classP, classType], 1)
        uniqueClass = prediction[:, -1].unique()
        # 对每一类分别进行非极大值抑制
        for uClassType in uniqueClass:
            tPrediction = prediction[prediction[:, -1] == uClassType]
            # if tPrediction.size(0) == 1:
            #     continue
            res = nms(tPrediction[:,:4], tPrediction[:,4] * tPrediction[:,5], threshold=0.3)
            # res = torchvision.ops.nms(tPrediction[:,:4], tPrediction[:,4] * tPrediction[:,5], 0.3) 这是torch自带的的nms
            tPrediction = tPrediction[res]
            output[i] = tPrediction if output[i] is None else torch.cat([output[i], tPrediction])
    return output


In [ ]:
def getTrueBox(outputs, bboxes):
    res = []
    for i,output in enumerate(outputs):
        # 对于一张图
        if output is None: # 没有预测框就跳过
            continue
        preBoxes = output[:,:4]
        preLabels = output[:,6]
        preConf = output[:,4] * output[:,5]
        targetBboxes = bboxes[bboxes[:,0] == i]
        targetBoxes = targetBboxes[:,2:]
        targetLabels = targetBboxes[:,1]
        detectedBox = []
        isCor = torch.zeros_like(preLabels)
        for j, preBox in enumerate(preBoxes):
            # 对于一个框
            if (len(detectedBox) == len(targetLabels)):
                break
            # print(iou(preBox, targetBoxes, isleftT2rightD=True))
            iout, maxI = torch.max(iou(preBox, targetBoxes, isleftT2rightD=True), 0)
            if iout > CONST.valIOUTher and maxI not in detectedBox and preLabels[j] == targetLabels[maxI]:
                isCor[j] = 1
                detectedBox.append(maxI)
        res.append([isCor, preConf, preLabels])
    return 

In [ ]:
def handleBox(input, yolo):
    outputs = []
    lastLayers = yolo.lastLayers
    for i, lasterLayer in enumerate(lastLayers):
        b, c, h, w = input[i].shape
        res = input[i].view(b, len(lasterLayer.anchor), -1, h, w).permute(0, 1, 3, 4, 2).contiguous()
        res[...,[0,1,4]] = res[...,[0,1,4]].sigmoid()
        res[...,5:] = res[...,5:].sigmoid()
        res[...,:2] = res[...,:2] + lasterLayer.grid[..., :2]
        res[...,2:4] = torch.exp(res[...,2:4]) * lasterLayer.grid[..., 2:4]
        res[...,:4] = res[...,:4] / torch.Tensor([w,h,w,h]).to(CONST.device)
        res = res.view(b,-1,5+CONST.classNumber)    #classNumber需要改为21？
        outputs.append(res)
    return torch.cat(outputs,1)

In [ ]:
def refer(input, imageSize, netInputSize = (416,416), isHold = False):
    if not isHold:
        for batchNum, val in  enumerate(input):
            val[...,:4] = val[...,:4] * torch.Tensor(imageSize).repeat((1,2)).to(CONST.device)
            input[batchNum] = val
    return input

In [ ]:
def valid():
    yolo = MyYOLO() # type: nn.Module
    getWeight(yolo)
    yolo.eval()
    yolo.to(CONST.device)
 
    flag=0
    with torch.no_grad():
        for imgs, imgsid in tqdm(testDataLoader, desc="Validating"):
            imgs = imgs.to(CONST.device)
            output = yolo(imgs)
            output = handleBox(output, yolo) # 处理先验框 返回的数据大小为(batchSize, 10647， 85)
            output = notMaxSuppression(output) # 非极大值抑制, 返回的数据为batchSize[x,y,x,y,conf,cls]    #形状有点问题
            # output = refer(output, Image.open(imgs).convert("RGB").size) # 将图片映射到原图坐标

            #将图片序号与output连接
            temp=torch.cat((imgsid,output),1)
            if flag==0:
                flag=1
                pred=temp
            else:
                pred=torch.cat((pred,temp),0)
    
    pred_list=pred.tolist()
    dim0, dim1 = pred.shape
    #将结果pred写回csv
    with open('test.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        for i in range(dim0):
            temp_list=[]
            for j in range(1,dim1):
                temp_list.append('{')
                a=",".join(pred_list[i][j])
                temp_list.append(a)
                temp_list.append('}')
            row_list=[pred[i][0]]           #picnum
            row_list.append("".join(temp_list))    #[picnum,{...}{...}]
            writer.writerow(row_list) 
    

In [ ]:
# def predict(img, isShow=True):
#     rawImg = Image.open(img).convert("RGB")
#     img = np.array(rawImg)
#     img = iaa.Sequential([
#         iaa.Resize((416,416))
#     ])(image=img)
#     img = np.transpose(img / 255, (2, 0, 1))
#     with torch.no_grad():
#         img = torch.from_numpy(img).unsqueeze(0).type(torch.float).to(CONST.device)
#         yolo = MyYOLO().to(CONST.device) # type: torch.nn.Module
#         # 迁移学习
#         getWeight(yolo)
#         # 预测图片
#         output = yolo(img) # 返回的数据大小为[(batchSize, 255,13,13), (batchSize, 255,13,13)]
#         output = handleBox(output, yolo) # 处理先验框 返回的数据大小为(batchSize, 10647， 85)
#         output = notMaxSuppression(output) # 非极大值抑制

#         print(f"抑制后的结果:{output[0].shape}")
#         output = refer(output, rawImg.size) # 将图片映射到原图坐标
#         output = output[0].to("cpu")
#         if len(output) == 0:
#             print("没有找到特征")
#             exit()
#         if isShow:
#             showPic(output, rawImg)
#         return output